### Using the TimeDB REST API

This notebook demonstrates the REST API for reading and writing time series data:
1. Setting up the database and starting the API server
2. Creating series and inserting data via `POST /upload`
3. Reading data via `GET /values` (flat and overlapping modes)
4. Updating records via `PUT /values`

In [1]:
from timedb import TimeDataClient
import pandas as pd
import requests
import json
from datetime import datetime, timezone, timedelta

td = TimeDataClient()
API_BASE_URL = "http://127.0.0.1:8000"
headers = {"Content-Type": "application/json"}
print("Ready")

Ready


## Part 1: Setup

Create the database schema via SDK (admin task — the API cannot create/delete schemas).

In [2]:
# Delete existing schema (optional - only if you want to start fresh)
# Uncomment the line below if you want to start with a clean database
td.delete()

# Create database schema
td.create()

Creating database schema...
✓ Schema created successfully


## Part 2: Start the API Server

Start the server before making API calls. In a notebook we run it as a background process.

In [3]:
# Start the API server in a separate terminal:
# timedb api --host 127.0.0.1 --port 8000

# Or using subprocess (for notebook use):
import subprocess
import time

# Kill any existing API server
subprocess.run(["pkill", "-f", "uvicorn.*timedb"], capture_output=True)
time.sleep(1)

# Start API server in background
process = subprocess.Popen(
    ["timedb", "api", "--host", "127.0.0.1", "--port", "8000"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)
time.sleep(3)  # Wait for server to start

# Check if API is running
try:
    response = requests.get(f"{API_BASE_URL}/")
    print("✓ API is running")
    print(f"  Name: {response.json()['name']}")
    print(f"  Version: {response.json().get('version', 'unknown')}")
except Exception as e:
    print(f"❌ API not running: {e}")

✓ API is running
  Name: TimeDB API
  Version: 0.1.1


## Part 3: Insert Data Using the API

Now let's create some sample time series data and insert it using the REST API.

In [4]:
# First, create the time series using the /series endpoint
# Use data_class='overlapping' so we can demonstrate updates later
# (updates only work on overlapping, not flat)
series_to_create = [
    {
        "name": "temperature",
        "description": "Temperature measurements in Celsius",
        "unit": "celsius",
        "labels": {"location": "office"},
        "data_class": "overlapping"
    },
    {
        "name": "humidity",
        "description": "Relative humidity percentage",
        "unit": "percent",
        "labels": {"location": "office"},
        "data_class": "overlapping"
    }
]

created_series = {}
for series_info in series_to_create:
    response = requests.post(
        f"{API_BASE_URL}/series",
        json=series_info,
        headers=headers
    )
    response.raise_for_status()
    result = response.json()
    series_name = series_info["name"]
    created_series[series_name] = result["series_id"]
    print(f"✓ Created series '{series_name}': {result['series_id']}")
    print(f"  Message: {result['message']}")

print(f"\n✓ Created {len(created_series)} time series")

✓ Created series 'temperature': 1
  Message: Series created successfully
✓ Created series 'humidity': 2
  Message: Series created successfully

✓ Created 2 time series


In [5]:
# Create sample time series data
base_time = datetime(2025, 1, 1, 0, 0, tzinfo=timezone.utc)
dates = [base_time + timedelta(hours=i) for i in range(24)]

# Prepare request payload for API
# Include series_id to reference the pre-created series (avoids duplicate creation)
value_rows = []
for i, date in enumerate(dates):
    # Add temperature value
    value_rows.append({
        "valid_time": date.isoformat(),
        "value_key": "temperature",
        "series_id": created_series["temperature"],
        "value": 20.0 + i * 0.3  # Temperature rising
    })
    # Add humidity value
    value_rows.append({
        "valid_time": date.isoformat(),
        "value_key": "humidity",
        "series_id": created_series["humidity"],
        "value": 60.0 - i * 0.5  # Humidity decreasing
    })

# Create batch request with batch_start_time
create_batch_request = {
    "batch_start_time": datetime.now(timezone.utc).isoformat(),
    "value_rows": value_rows
}

print(f"Prepared {len(value_rows)} value rows to insert")
print(f"Time range: {dates[0]} to {dates[-1]}")
print(f"Series: {', '.join(created_series.keys())}")

Prepared 48 value rows to insert
Time range: 2025-01-01 00:00:00+00:00 to 2025-01-01 23:00:00+00:00
Series: temperature, humidity


### 3.1: Upload the Data

Now let's upload the time series data using the `/upload` endpoint.

In [6]:
# Upload data via API
response = requests.post(
    f"{API_BASE_URL}/upload",
    json=create_batch_request,
    headers=headers
)
response.raise_for_status()

result = response.json()
print(f"✓ Created batch with ID: {result['batch_id']}")
print(f"  Message: {result['message']}")
print(f"\nSeries IDs returned:")
for series_name, series_id in result['series_ids'].items():
    print(f"  {series_name}: {series_id}")

# Store batch_id and series_ids for later use
batch_id = result['batch_id']
series_ids = result['series_ids']  # Maps series name -> series_id

✓ Created batch with ID: 1
  Message: Batch created successfully

Series IDs returned:
  temperature: 1
  humidity: 2


### 3.2: List All Time Series

After uploading data, you can list all available time series to see their metadata.

In [7]:
# List all time series
response = requests.get(f"{API_BASE_URL}/list_timeseries", headers=headers)
response.raise_for_status()

timeseries_list = response.json()
print(f"✓ Found {len(timeseries_list)} time series")
print("\nSeries information:")
for series_id, series_info in timeseries_list.items():
    print(f"  {series_id}:")
    print(f"    Name: {series_info['name']}")
    print(f"    Description: {series_info.get('description', 'N/A')}")
    print(f"    Unit: {series_info['unit']}")
    print(f"    Labels: {series_info.get('labels', {})}")

✓ Found 2 time series

Series information:
  2:
    Name: humidity
    Description: Relative humidity percentage
    Unit: percent
    Labels: {'location': 'office'}
  1:
    Name: temperature
    Description: Temperature measurements in Celsius
    Unit: celsius
    Labels: {'location': 'office'}


## Part 4: Read Data Using the API

Let's read the time series data we just inserted using the API.

In [8]:
# Read data via API
params = {
    "start_valid": base_time.isoformat(),
    "end_valid": (base_time + timedelta(hours=24)).isoformat(),
    "mode": "flat",  # "flat" returns latest known_time per valid_time
}

response = requests.get(f"{API_BASE_URL}/values", params=params, headers=headers)
response.raise_for_status()

data = response.json()
print(f"✓ Retrieved {data['count']} records via API")

# Convert to DataFrame for easier viewing
if data['count'] > 0:
    df_api = pd.DataFrame(data['data'])
    # Convert ISO strings back to datetime
    df_api['valid_time'] = pd.to_datetime(df_api['valid_time'])
    print("\nFirst few rows:")
    print(df_api.head(10))
    print(f"\nDataFrame shape: {df_api.shape}")
    print(f"Columns: {list(df_api.columns)}")
else:
    print("No data found")

✓ Retrieved 48 records via API

First few rows:
                 valid_time  series_id  value         name     unit  \
0 2025-01-01 00:00:00+00:00          1   20.0  temperature  celsius   
1 2025-01-01 00:00:00+00:00          2   60.0     humidity  percent   
2 2025-01-01 01:00:00+00:00          1   20.3  temperature  celsius   
3 2025-01-01 01:00:00+00:00          2   59.5     humidity  percent   
4 2025-01-01 02:00:00+00:00          1   20.6  temperature  celsius   
5 2025-01-01 02:00:00+00:00          2   59.0     humidity  percent   
6 2025-01-01 03:00:00+00:00          1   20.9  temperature  celsius   
7 2025-01-01 03:00:00+00:00          2   58.5     humidity  percent   
8 2025-01-01 04:00:00+00:00          1   21.2  temperature  celsius   
9 2025-01-01 04:00:00+00:00          2   58.0     humidity  percent   

                   labels  
0  {'location': 'office'}  
1  {'location': 'office'}  
2  {'location': 'office'}  
3  {'location': 'office'}  
4  {'location': 'office'}  
5 

### 4.1: Read with Different Modes

The API supports two query modes:
- **"flat"**: Returns the latest value per (valid_time, series_id), determined by most recent known_time
- **"overlapping"**: Returns all forecast revisions with their known_time, useful for backtesting

Let's try the overlapping mode:

In [9]:
# Read in overlapping mode to see all forecast revisions
params_overlapping = {
    "start_valid": base_time.isoformat(),
    "end_valid": (base_time + timedelta(hours=6)).isoformat(),  # Smaller range for clarity
    "mode": "overlapping",  # This mode shows all known_time revisions
}

response = requests.get(f"{API_BASE_URL}/values", params=params_overlapping, headers=headers)
response.raise_for_status()

data_overlapping = response.json()
print(f"✓ Retrieved {data_overlapping['count']} records in overlapping mode")

if data_overlapping['count'] > 0:
    df_overlapping = pd.DataFrame(data_overlapping['data'])
    df_overlapping['valid_time'] = pd.to_datetime(df_overlapping['valid_time'])
    if 'known_time' in df_overlapping.columns:
        df_overlapping['known_time'] = pd.to_datetime(df_overlapping['known_time'])
    print("\nFirst few rows (showing forecast revisions):")
    print(df_overlapping.head(10))

✓ Retrieved 12 records in overlapping mode

First few rows (showing forecast revisions):
                        known_time                valid_time  series_id  \
0 2026-02-11 15:28:43.451586+00:00 2025-01-01 00:00:00+00:00          1   
1 2026-02-11 15:28:43.451586+00:00 2025-01-01 00:00:00+00:00          2   
2 2026-02-11 15:28:43.451586+00:00 2025-01-01 01:00:00+00:00          1   
3 2026-02-11 15:28:43.451586+00:00 2025-01-01 01:00:00+00:00          2   
4 2026-02-11 15:28:43.451586+00:00 2025-01-01 02:00:00+00:00          1   
5 2026-02-11 15:28:43.451586+00:00 2025-01-01 02:00:00+00:00          2   
6 2026-02-11 15:28:43.451586+00:00 2025-01-01 03:00:00+00:00          1   
7 2026-02-11 15:28:43.451586+00:00 2025-01-01 03:00:00+00:00          2   
8 2026-02-11 15:28:43.451586+00:00 2025-01-01 04:00:00+00:00          1   
9 2026-02-11 15:28:43.451586+00:00 2025-01-01 04:00:00+00:00          2   

   value         name     unit                  labels  
0   20.0  temperature  celsi

## Part 5: Update Records Using the API

Updates create a new version with a new `known_time` while preserving the original for audit trail. Requires `batch_id`, `valid_time`, and `series_id`.

In [10]:
# Update a record via the API
update_request = {
    "updates": [
        {
            "batch_id": batch_id,
            "valid_time": base_time.isoformat(),
            "series_id": created_series["temperature"],
            "value": 22.5,
            "annotation": "Updated via API"
        }
    ]
}

response = requests.put(f"{API_BASE_URL}/values", json=update_request, headers=headers)
response.raise_for_status()
update_result = response.json()
print(f"Updated {len(update_result['updated'])} record(s), skipped {len(update_result['skipped_no_ops'])} no-op(s)")

# Verify the update
params_verify = {
    "start_valid": base_time.isoformat(),
    "end_valid": (base_time + timedelta(hours=1)).isoformat(),
    "mode": "flat"
}
response = requests.get(f"{API_BASE_URL}/values", params=params_verify, headers=headers)
response.raise_for_status()
data_verify = response.json()

if data_verify['count'] > 0:
    df_verify = pd.DataFrame(data_verify['data'])
    temp_row = df_verify[df_verify['name'] == 'temperature'].iloc[0]
    print(f"Verified: temperature at {base_time} is now {temp_row['value']} (was 20.0)")

Updated 1 record(s), skipped 0 no-op(s)
Verified: temperature at 2025-01-01 00:00:00+00:00 is now 22.5 (was 20.0)


### Summary

**Key Endpoints:**
- `POST /series` — create a time series
- `POST /upload` — insert a batch of values (pass `series_id` to reference existing series)
- `GET /values` — read values (`mode=flat` for latest, `mode=overlapping` for all revisions)
- `PUT /values` — update existing overlapping records (creates new version)
- `GET /list_timeseries` — list all series metadata

**Starting the server:**
```bash
timedb api --host 127.0.0.1 --port 8000
```